In [1]:
import open3d as o3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Define functions

In [4]:
def visualize_point_cloud(nome_point_cloud):
    pcd = o3d.io.read_point_cloud(nome_point_cloud, format = 'ply')
    print(np.asarray(pcd.points))
    o3d.visualization.draw_geometries([pcd])
    return pcd

In [5]:
def display_inlier_outlier(cloud, ind):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert = True)

    #print("Showing outliers (red) and inliers (gray): ")
    #outlier_cloud.paint_uniform_color([1, 0, 0])
    #inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    o3d.visualization.draw_geometries([inlier_cloud])
    #print("inlier_cloud: ", inlier_cloud)

In [6]:
def save_point_cloud(nome_file, punti_utili, colori_utili):
    file = open(nome_file,"w")
    file.write("ply \n")
    file.write("format ascii 1.0 \n")
    file.write("element vertex " + str(punti_utili.shape[0]) + "\n")
    file.write("property float x \n")
    file.write("property float y \n")
    file.write("property float z \n")
    file.write("property uchar red \n")
    file.write("property uchar green \n")
    file.write("property uchar blue \n")
    file.write("end_header \n")
    i = 0
    j = 0
    x = 0
    for i in range(np.shape(punti_utili)[0]):
        for j in range (3):
            file.write(str(punti_utili[i, j]) + " ")
        for x in range (3):
            file.write(str(colori_utili[i, x]) + " ")
        file.write("\n")

In [7]:
#Modificare il valore di nb_neighbors per specificare quanti vicini devono essere presi in considerazione per calcolare la distanza media per un dato punto
#Modificare il valore di std_ratio in base a quanto aggressiva si vuole la funzione, rispettivamente più è basso il valore più outlier verranno eliminati 
def statistical_outlier_removal(pcd, nb_neighbors, std_ratio):
    print("Statistical outlier removal")
    cl, ind = pcd.remove_statistical_outlier(nb_neighbors, std_ratio)
    display_inlier_outlier(pcd, ind)
    return cl

In [8]:
#Modificare il valore di nb_points per selezionare la quantità minima di punti che la sfera deve contenere
#Modificare il valore di radius per definire il raggio della sfera che verrà utilizzata per il conteggio dei vicini
def radius_outlier_removal(pcd, nb_points, radius):
    print("Radius outlier removal")
    cl, ind = pcd.remove_radius_outlier(nb_points, radius)
    display_inlier_outlier(pcd, ind)
    return cl

# Point cloud cleaning

In [7]:
pcd = visualize_point_cloud("nike.ply")

[[-2.94941  -1.62554   3.79955 ]
 [-1.59964  -2.15609   0.564713]
 [-1.08616  -1.23756   5.98223 ]
 ...
 [ 0.691917  0.597042  4.80825 ]
 [ 0.476538  0.124745  4.2401  ]
 [ 0.799901 -0.171617  4.30556 ]]


In [81]:
min_x = -3.0
max_x = 8.0
min_y = -3.0
max_y = 3.0
min_z = -3.0
max_z = 8.0
#filtra per posizione in modo tale da restringere il campo di interesse della point cloud
filtered_pcd = np.where((np.asarray(pcd.points)[:,0] < max_x) & (np.asarray(pcd.points)[:,0] > min_x) & 
                        (np.asarray(pcd.points)[:,1] < max_y) & (np.asarray(pcd.points)[:,1] > min_y) &
                        (np.asarray(pcd.points)[:,2] < max_z) & (np.asarray(pcd.points)[:,1] > min_z))
print(filtered_pcd)
print(np.shape(filtered_pcd))

(array([     0,      1,      2, ..., 287456, 287457, 287458], dtype=int64),)
(1, 257805)


In [82]:
punti_utili = np.zeros((np.shape(filtered_pcd)[1], 3), dtype = float)
colori_utili = np.zeros((np.shape(filtered_pcd)[1], 3), dtype = float)
print(np.shape(punti_utili))
print(np.shape(colori_utili))

(257805, 3)
(257805, 3)


In [83]:
#filtra i punti e li salva in una nuova matrice
i = 0
j = 0
x = 0
for i in range(np.shape(np.asarray(pcd.points))[0]):
    if ((np.asarray(pcd.points)[i,0] < max_x) & (np.asarray(pcd.points)[i,0] > min_x) & 
        (np.asarray(pcd.points)[i,1] < max_y) & (np.asarray(pcd.points)[i,1] > min_y) &
        (np.asarray(pcd.points)[i,2] < max_z) & (np.asarray(pcd.points)[i,1] > min_z)):
        for j in range(3):
            punti_utili[x,j] = np.asarray(pcd.points)[i,j]
        x = x + 1
print(punti_utili)

[[-2.94941  -1.62554   3.79955 ]
 [-1.59964  -2.15609   0.564713]
 [-1.08616  -1.23756   5.98223 ]
 ...
 [ 0.691917  0.597042  4.80825 ]
 [ 0.476538  0.124745  4.2401  ]
 [ 0.799901 -0.171617  4.30556 ]]


In [84]:
#crea una nuova matrice con i colori dei punti che non nel campo di interesse e li riporta in scala [0, 255]
i = 0
j = 0
x = 0
for i in range(np.shape(np.asarray(pcd.points))[0]):
    if ((np.asarray(pcd.points)[i,0] < max_x) & (np.asarray(pcd.points)[i,0] > min_x) & 
        (np.asarray(pcd.points)[i,1] < max_y) & (np.asarray(pcd.points)[i,1] > min_y) &
        (np.asarray(pcd.points)[i,2] < max_z) & (np.asarray(pcd.points)[i,1] > min_z)):
        for j in range(3):
            colori_utili[x,j] = np.asarray(pcd.colors)[i,j] * 255
        x = x + 1
colori_utili = colori_utili.astype(int)
print(colori_utili)

[[  0 255   0]
 [  0 255   0]
 [  0 255   0]
 ...
 [134 141 144]
 [255  73   2]
 [181  42   2]]


In [85]:
save_point_cloud("pointCloud_PrimaModifica_nike.ply", punti_utili, colori_utili)

In [8]:
pcd_first_modify = visualize_point_cloud("pointCloud_PrimaModifica_nike.ply")

[[-2.94941  -1.62554   3.79955 ]
 [-1.59964  -2.15609   0.564713]
 [-1.08616  -1.23756   5.98223 ]
 ...
 [ 0.691917  0.597042  4.80825 ]
 [ 0.476538  0.124745  4.2401  ]
 [ 0.799901 -0.171617  4.30556 ]]


In [8]:
# property float x
print(max(np.asarray(pcd.points)[:,0]))
print(min(np.asarray(pcd.points)[:,0]))

23.1878
-27.6227


In [9]:
# property float y
print(max(np.asarray(pcd.points)[:,1]))
print(min(np.asarray(pcd.points)[:,1]))

12.6237
-11.5138


In [10]:
# property float z
print(max(np.asarray(pcd.points)[:,2]))
print(min(np.asarray(pcd.points)[:,2]))

31.3765
-24.3987


In [87]:
green_value = 1
#elimina le telecamere che sono completamente verdi
filtered_pcd_green_less = np.where((np.asarray(pcd_first_modify.colors)[:,1] != green_value))
print(filtered_pcd_green_less)
print(np.shape(filtered_pcd_green_less))

(array([  2632,   2633,   2634, ..., 257802, 257803, 257804], dtype=int64),)
(1, 254803)


In [88]:
punti_utili_green_less = np.zeros((np.shape(filtered_pcd_green_less)[1], 3), dtype = float)
colori_utili_green_less = np.zeros((np.shape(filtered_pcd_green_less)[1], 3), dtype = float)
print(np.shape(punti_utili_green_less))
print(np.shape(colori_utili_green_less))

(254803, 3)
(254803, 3)


In [89]:
#filtra i punti e li salva in una nuova matrice
i = 0
j = 0
x = 0
for i in range(np.shape(np.asarray(pcd_first_modify.points))[0]):
    if (np.asarray(pcd_first_modify.colors)[i,1] != green_value):
        for j in range(3):
            punti_utili_green_less[x,j] = np.asarray(pcd_first_modify.points)[i,j]
        x = x + 1
print(punti_utili_green_less)

[[ 1.09312  -0.451514  3.88326 ]
 [ 0.997504  0.308511  3.75567 ]
 [ 1.12951  -0.148667  3.81329 ]
 ...
 [ 0.691917  0.597042  4.80825 ]
 [ 0.476538  0.124745  4.2401  ]
 [ 0.799901 -0.171617  4.30556 ]]


In [90]:
#crea una nuova matrice con i colori dei punti che non sono telecamere e li riporta in scala [0, 255]
i = 0
j = 0
x = 0
for i in range(np.shape(np.asarray(pcd_first_modify.points))[0]):
    if (np.asarray(pcd_first_modify.colors)[i,1] != green_value):
        for j in range(3):
            colori_utili_green_less[x,j] = np.asarray(pcd_first_modify.colors)[i,j] * 255
        x = x + 1
colori_utili_green_less = colori_utili_green_less.astype(int)
print(colori_utili_green_less)

[[216 160 146]
 [ 68  11   0]
 [  7   0   0]
 ...
 [134 141 144]
 [255  73   2]
 [181  42   2]]


In [91]:
save_point_cloud("pointCloud_SecondaModifica_nike.ply", punti_utili_green_less, colori_utili_green_less)

In [9]:
pcd_second_modify = visualize_point_cloud("pointCloud_SecondaModifica_nike.ply")

[[ 1.09312  -0.451514  3.88326 ]
 [ 0.997504  0.308511  3.75567 ]
 [ 1.12951  -0.148667  3.81329 ]
 ...
 [ 0.691917  0.597042  4.80825 ]
 [ 0.476538  0.124745  4.2401  ]
 [ 0.799901 -0.171617  4.30556 ]]
[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.


In [8]:
cl_statical = statistical_outlier_removal(pcd_second_modify, 20, 2.0)

Statistical outlier removal
Showing outliers (red) and inliers (gray): 


In [9]:
np.shape(np.asarray(cl_statical.points))

(250407, 3)

In [10]:
cl_statical2 = statistical_outlier_removal(cl_statical, 20, 2.0)

Statistical outlier removal
Showing outliers (red) and inliers (gray): 


In [11]:
np.shape(np.asarray(cl_statical2.points))

(234675, 3)

In [14]:
print(np.asarray(cl_statical2.points))
print(" ")
cl_statical2_colori = np.asarray(cl_statical2.colors) * 255
cl_statical2_colori = cl_statical2_colori.astype(int)
print(cl_statical2_colori)

[[ 1.09312  -0.451514  3.88326 ]
 [ 0.997504  0.308511  3.75567 ]
 [ 1.12951  -0.148667  3.81329 ]
 ...
 [ 0.010718  0.229622  4.52808 ]
 [ 0.476538  0.124745  4.2401  ]
 [ 0.799901 -0.171617  4.30556 ]]
 
[[216 160 146]
 [ 68  11   0]
 [  7   0   0]
 ...
 [234  50   0]
 [255  73   2]
 [181  42   2]]


In [15]:
save_point_cloud("pointCloud_TerzaModifica_nike.ply", np.asarray(cl_statical2.points), cl_statical2_colori)

In [10]:
pcd_third_modify = visualize_point_cloud("pointCloud_TerzaModifica_nike.ply")

[[ 1.09312  -0.451514  3.88326 ]
 [ 0.997504  0.308511  3.75567 ]
 [ 1.12951  -0.148667  3.81329 ]
 ...
 [ 0.010718  0.229622  4.52808 ]
 [ 0.476538  0.124745  4.2401  ]
 [ 0.799901 -0.171617  4.30556 ]]


In [18]:
range_value = 0.019
#filtra i colori grigi
filtered_pcd_gray_less = np.where((np.asarray(pcd_third_modify.colors)[:,0] > ((np.asarray(pcd_third_modify.colors)[:,1]) + range_value)) | 
                                  (np.asarray(pcd_third_modify.colors)[:,0] < ((np.asarray(pcd_third_modify.colors)[:,1]) - range_value)) & 
                                  (np.asarray(pcd_third_modify.colors)[:,0] > ((np.asarray(pcd_third_modify.colors)[:,2]) + range_value)) | 
                                  (np.asarray(pcd_third_modify.colors)[:,0] < ((np.asarray(pcd_third_modify.colors)[:,2]) - range_value)))
print(filtered_pcd_gray_less)
print(np.shape(filtered_pcd_gray_less))

(array([     0,      1,      2, ..., 234672, 234673, 234674], dtype=int64),)
(1, 230899)


In [19]:
punti_utili_gray_less = np.zeros((np.shape(filtered_pcd_gray_less)[1], 3), dtype = float)
colori_utili_gray_less = np.zeros((np.shape(filtered_pcd_gray_less)[1], 3), dtype = float)
print(np.shape(punti_utili_gray_less))
print(np.shape(colori_utili_gray_less))

(230899, 3)
(230899, 3)


In [20]:
#filtra i punti e li salva in una nuova matrice
i = 0
j = 0
x = 0
for i in range(np.shape(np.asarray(pcd_third_modify.points))[0]):
    if ((np.asarray(pcd_third_modify.colors)[i,0] > ((np.asarray(pcd_third_modify.colors)[i,1]) + range_value)) | 
        (np.asarray(pcd_third_modify.colors)[i,0] < ((np.asarray(pcd_third_modify.colors)[i,1]) - range_value)) & 
        (np.asarray(pcd_third_modify.colors)[i,0] > ((np.asarray(pcd_third_modify.colors)[i,2]) + range_value)) | 
        (np.asarray(pcd_third_modify.colors)[i,0] < ((np.asarray(pcd_third_modify.colors)[i,2]) - range_value))):
        for j in range(3):
            punti_utili_gray_less[x,j] = np.asarray(pcd_third_modify.points)[i,j]
        x = x + 1
print(punti_utili_gray_less)

[[ 1.09312  -0.451514  3.88326 ]
 [ 0.997504  0.308511  3.75567 ]
 [ 1.12951  -0.148667  3.81329 ]
 ...
 [ 0.010718  0.229622  4.52808 ]
 [ 0.476538  0.124745  4.2401  ]
 [ 0.799901 -0.171617  4.30556 ]]


In [21]:
#crea una nuova matrice con i colori dei punti che non superano la soglia e li riporta in scala [0, 255]
i = 0
j = 0
x = 0
for i in range(np.shape(np.asarray(pcd_third_modify.points))[0]):
    if ((np.asarray(pcd_third_modify.colors)[i,0] > ((np.asarray(pcd_third_modify.colors)[i,1]) + range_value)) | 
        (np.asarray(pcd_third_modify.colors)[i,0] < ((np.asarray(pcd_third_modify.colors)[i,1]) - range_value)) & 
        (np.asarray(pcd_third_modify.colors)[i,0] > ((np.asarray(pcd_third_modify.colors)[i,2]) + range_value)) | 
        (np.asarray(pcd_third_modify.colors)[i,0] < ((np.asarray(pcd_third_modify.colors)[i,2]) - range_value))):
        for j in range(3):
            colori_utili_gray_less[x,j] = np.asarray(pcd_third_modify.colors)[i,j] * 255
        x = x + 1
colori_utili_gray_less = colori_utili_gray_less.astype(int)
print(colori_utili_gray_less)

[[216 160 146]
 [ 68  11   0]
 [  7   0   0]
 ...
 [234  50   0]
 [255  73   2]
 [181  42   2]]


In [22]:
save_point_cloud("pointCloud_GrayLess_nike.ply", punti_utili_gray_less, colori_utili_gray_less)

In [11]:
pcd_gray_less = visualize_point_cloud("pointCloud_GrayLess_nike.ply")

[[ 1.09312  -0.451514  3.88326 ]
 [ 0.997504  0.308511  3.75567 ]
 [ 1.12951  -0.148667  3.81329 ]
 ...
 [ 0.010718  0.229622  4.52808 ]
 [ 0.476538  0.124745  4.2401  ]
 [ 0.799901 -0.171617  4.30556 ]]


In [16]:
blue_value = 0.66
#filtra i colori sul blu
filtered_pcd_blue_less = np.where(((np.asarray(pcd_gray_less.colors)[:,0]) > (np.asarray(pcd_gray_less.colors)[:,2])) &
                                  ((np.asarray(pcd_gray_less.colors)[:,1]) > (np.asarray(pcd_gray_less.colors)[:,2])) &
                                  ((np.asarray(pcd_gray_less.colors)[:,2]) < blue_value))
print(filtered_pcd_blue_less)
print(np.shape(filtered_pcd_blue_less))

(array([     0,      1,      4, ..., 230896, 230897, 230898], dtype=int64),)
(1, 189954)


In [17]:
punti_utili_blue_less = np.zeros((np.shape(filtered_pcd_blue_less)[1], 3), dtype = float)
colori_utili_blue_less = np.zeros((np.shape(filtered_pcd_blue_less)[1], 3), dtype = float)
print(np.shape(punti_utili_blue_less))
print(np.shape(colori_utili_blue_less))

(189954, 3)
(189954, 3)


In [18]:
#filtra i punti e li salva in una nuova matrice
i = 0
j = 0
x = 0
for i in range(np.shape(np.asarray(pcd_gray_less.points))[0]):
    if (((np.asarray(pcd_gray_less.colors)[i,0]) > (np.asarray(pcd_gray_less.colors)[i,2])) &
        ((np.asarray(pcd_gray_less.colors)[i,1]) > (np.asarray(pcd_gray_less.colors)[i,2])) &
        ((np.asarray(pcd_gray_less.colors)[i,2]) < blue_value)):
        for j in range(3):
            punti_utili_blue_less[x,j] = np.asarray(pcd_gray_less.points)[i,j]
        x = x + 1
print(punti_utili_blue_less)

[[ 1.09312  -0.451514  3.88326 ]
 [ 0.997504  0.308511  3.75567 ]
 [-0.977566 -0.800794  2.7224  ]
 ...
 [ 0.010718  0.229622  4.52808 ]
 [ 0.476538  0.124745  4.2401  ]
 [ 0.799901 -0.171617  4.30556 ]]


In [19]:
#crea una nuova matrice con i colori dei punti che non superano la soglia e li riporta in scala [0, 255]
i = 0
j = 0
x = 0
for i in range(np.shape(np.asarray(pcd_gray_less.points))[0]):
    if (((np.asarray(pcd_gray_less.colors)[i,0]) > (np.asarray(pcd_gray_less.colors)[i,2])) &
        ((np.asarray(pcd_gray_less.colors)[i,1]) > (np.asarray(pcd_gray_less.colors)[i,2])) &
        ((np.asarray(pcd_gray_less.colors)[i,2]) < blue_value)):
        for j in range(3):
            colori_utili_blue_less[x,j] = np.asarray(pcd_gray_less.colors)[i,j] * 255
        x = x + 1
colori_utili_blue_less = colori_utili_blue_less.astype(int)
print(colori_utili_blue_less)

[[216 160 146]
 [ 68  11   0]
 [230 169 165]
 ...
 [234  50   0]
 [255  73   2]
 [181  42   2]]


In [22]:
save_point_cloud("pointCloud_BlueLess_nike.ply", punti_utili_blue_less, colori_utili_blue_less)

In [10]:
pcd_blue_less = visualize_point_cloud("pointCloud_BlueLess_nike.ply")

[[ 1.09312  -0.451514  3.88326 ]
 [ 0.997504  0.308511  3.75567 ]
 [-0.977566 -0.800794  2.7224  ]
 ...
 [ 0.010718  0.229622  4.52808 ]
 [ 0.476538  0.124745  4.2401  ]
 [ 0.799901 -0.171617  4.30556 ]]
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: L'operazione di trasformazione richiesta non è supportata. 


In [11]:
cl_statical3 = statistical_outlier_removal(pcd_blue_less, 80, 2.0)

Statistical outlier removal
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: L'operazione di trasformazione richiesta non è supportata. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Handle non valido. 


In [27]:
np.shape(np.asarray(cl_statical3.points))

(181632, 3)

In [12]:
cl_statical4 = statistical_outlier_removal(cl_statical3, 80, 2.0)

Statistical outlier removal
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: L'operazione di trasformazione richiesta non è supportata. 


In [140]:
#la funzione segment_plane permette di effettuare una plane segmentation
#tre parametri:
#distance_threshold definisce la distanza massima che un punto può avere da un piano stimato per essere considerato un inlier
#ransac_n definisce il numero di punti campionati casualmente per stimare un piano 
#num_iterations definisce la frequenza con cui un piano casuale viene campionato e verificato
#restituisce il piano come [a, b, c, d] tale che per ogni punto (x,y,z) sul piano abbiamo ax + by + cz + d = 0
#restituisce anche un elenco di indici dei punti inlier
plane_model, inliers = cl_statical4.segment_plane(distance_threshold = 0.5,
                                         ransac_n = 3,
                                         num_iterations = 1000)
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

inlier_cloud = cl_statical4.select_by_index(inliers)
#inlier_cloud.paint_uniform_color([0, 0, 1.0])
outlier_cloud = cl_statical4.select_by_index(inliers, invert=True)
o3d.visualization.draw_geometries([inlier_cloud])

Plane equation: 0.02x + 0.99y + -0.13z + 0.80 = 0


In [141]:
print(np.asarray(inlier_cloud.points))
print(" ")
inlier_cloud_colori = np.asarray(inlier_cloud.colors) * 255
inlier_cloud_colori = inlier_cloud_colori.astype(int)
print(inlier_cloud_colori)

[[ 1.09312  -0.451514  3.88326 ]
 [ 0.997504  0.308511  3.75567 ]
 [-0.977566 -0.800794  2.7224  ]
 ...
 [ 0.010718  0.229622  4.52808 ]
 [ 0.476538  0.124745  4.2401  ]
 [ 0.799901 -0.171617  4.30556 ]]
 
[[216 160 146]
 [ 68  11   0]
 [230 169 165]
 ...
 [234  50   0]
 [255  73   2]
 [181  42   2]]


In [142]:
save_point_cloud("pointCloud_Segmentation_nike6.ply", np.asarray(inlier_cloud.points), inlier_cloud_colori)

In [13]:
pcd_plane_segmentation = visualize_point_cloud("pointCloud_Segmentation_nike5.ply")

[[ 1.09312  -0.451514  3.88326 ]
 [ 0.997504  0.308511  3.75567 ]
 [-0.977566 -0.800794  2.7224  ]
 ...
 [ 0.010718  0.229622  4.52808 ]
 [ 0.476538  0.124745  4.2401  ]
 [ 0.799901 -0.171617  4.30556 ]]
The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


In [164]:
#crea una mesh dalla point cloud
alpha = 0.8

print(f"alpha = {alpha:.3f}")
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd_plane_segmentation, alpha)
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh], mesh_show_back_face = True)

alpha=0.800


In [165]:
#ricostruisce la point cloud dalla mesh
pcd_ricostruita = mesh.sample_points_uniformly(number_of_points = 200000)
o3d.visualization.draw_geometries([pcd_ricostruita])

In [166]:
print(np.asarray(pcd_ricostruita.points))
print(" ")
pcd_ricostruita_colori = np.asarray(pcd_ricostruita.colors) * 255
pcd_ricostruita_colori = pcd_ricostruita_colori.astype(int)
print(pcd_ricostruita_colori)

[[-0.09904709 -0.89138746  1.88103991]
 [-0.0908544  -0.88936895  1.87791749]
 [-0.10891099 -0.88796152  1.87819221]
 ...
 [-0.75243951 -0.74125284  3.40579063]
 [-0.79635714 -0.72782779  3.46419336]
 [-0.74380828 -0.74352598  3.39620058]]
 
[[246 125  83]
 [246 126  87]
 [248 117  69]
 ...
 [191 123  82]
 [229 120  57]
 [184 123  87]]


In [167]:
save_point_cloud("pointCloud_Ricostruita_nike.ply", np.asarray(pcd_ricostruita.points), pcd_ricostruita_colori)

In [9]:
pcd_finale = visualize_point_cloud("pointCloud_Ricostruita_nike.ply")

[[-0.09904709 -0.89138746  1.88103991]
 [-0.0908544  -0.88936895  1.87791749]
 [-0.10891099 -0.88796152  1.87819221]
 ...
 [-0.75243951 -0.74125284  3.40579063]
 [-0.79635714 -0.72782779  3.46419336]
 [-0.74380828 -0.74352598  3.39620058]]
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Handle non valido. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: L'operazione di trasformazione richiesta non è supportata. 


# Error estimate calculation

In [ ]:
#salva la mesh
o3d.io.write_triangle_mesh("mesh_nike.ply", meshNike)

In [2]:
#carica la mesh
mesh_ruotata_nike = o3d.io.read_triangle_mesh("NikeRuotata.ply")
o3d.visualization.draw_geometries([mesh_ruotata_nike], mesh_show_back_face = True)

In [3]:
#ricostruisce la point cloud dalla mesh
pcd_ricostruita_ruotata_nike = mesh_ruotata_nike.sample_points_uniformly(number_of_points = 200000)
o3d.visualization.draw_geometries([pcd_ricostruita_ruotata_nike])

In [4]:
#carica la mesh
mesh_scatola_nike = o3d.io.read_triangle_mesh("ScatolaNike.ply")
o3d.visualization.draw_geometries([mesh_scatola_nike], mesh_show_back_face=True)

In [5]:
#ricostruisce la point cloud dalla mesh
pcd_scatola_nike = mesh_scatola_nike.sample_points_uniformly(number_of_points = 200000)
o3d.visualization.draw_geometries([pcd_scatola_nike])

In [6]:
#calcola la distanza da una point cloud di origine a una point cloud di destinazione 
#calcola per ogni punto nella point cloud di origine la distanza dal punto più vicino nella point cloud di destinazione
dists_nike = pcd_ricostruita_ruotata_nike.compute_point_cloud_distance(pcd_scatola_nike)
dists_nike = np.asarray(dists_nike)
print(dists_nike)

[0.27853899 0.25198617 0.24331912 ... 0.01502436 0.01875736 0.01289454]


In [7]:
np.mean(dists_nike)

0.16542054202285508